In [1]:
import scraping
import pickle
import pandas as pd
import tqdm

import os
import sys
sys.path.append(os.pardir)
import common.com as common
from common import const,utils

# データ取得・保存処理
- netkeiba.comから指定された期間に開催された全レース結果を取得し、origin_race_refundテーブルに保存する

In [2]:
# 開催日一覧を取得
from_date = "2013-01"
to_date = "2023-12"
kaisai_date_list = scraping.scrape_kaisai_date(from_=from_date, to_=to_date)

  0%|          | 0/132 [00:00<?, ?it/s]

In [3]:
# 開催日に開かれたレースのレースID一覧を取得
race_id_list = scraping.scrape_race_id_list(kaisai_date_list)

  0%|          | 0/1196 [00:00<?, ?it/s]

In [4]:
# レースID一覧を保存
race_id_list_file_path=f"{const.RACE_ID_LIST_DIR}{from_date}_{to_date}.pickle"
with open(race_id_list_file_path, "wb") as f:
    pickle.dump(race_id_list, f)

In [5]:
# レースID一覧を読み込み
with open(race_id_list_file_path, 'rb') as file:
    race_id_list = pickle.load(file)


# レースID一覧からレースデータを取得
race_refund_datas=pd.DataFrame()
for race_id in tqdm.tqdm(race_id_list):
    data = scraping.scrape_race_refund(race_id)
    race_refund_datas = pd.concat([race_refund_datas,data])

100%|██████████| 38002/38002 [4:17:44<00:00,  2.46it/s]  


In [6]:
# レースデータをcsvに保存
df = pd.DataFrame(race_refund_datas)

df.columns = const.ORIGIN_RACE_REFUND_HEADERS
csv_file_path=race_id_list_file_path.replace('.pickle', '.csv').replace('race_id_lists', 'race_refund_datas')
df.to_csv(csv_file_path, index=False)